In [58]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import numpy as np
import pandas as pd
import cvxpy as cp
import torch
from copy import copy, deepcopy


In [59]:
n=5

In [60]:
np.random.rand()

0.15133666396337742

In [61]:
ds = []

In [62]:
a_min = 0.9
a_var = 0.1

b_min = 0.1
b_var = 0.1

r_min = 0.1
r_var = 0.1

x_0_min = 0.2
x_0_var = 0.1

x_ref_min = 0.7
x_ref_var = 0.1

In [63]:
for i in range(1000):
	
	a=0.9
	b=0.1
	r = r_var*np.random.rand() + r_min
	x_0=  x_0_var*np.random.rand() + x_0_min
	x_ref = 0.1*np.array([np.random.rand()]*(n)) + 0.7
	

	L = r * np.identity(n)
	I = np.identity(n)
	A = np.array([a**(i+1) for i in range(n)])
	B = np.array([[a**(i-1) * b for i in range(j, 0, -1)] + [0]*(n-j) for j in range(1, n+1)])

	u = cp.Variable(n)
	constraints = [u <= np.ones(n), -u <= np.zeros(n), (A*x_0 +B@u) <= np.ones(n), -(A*x_0 +B@u) <= np.zeros(n)]
	prob = cp.Problem(cp.Minimize(cp.quad_form(A*x_0 +B@u - x_ref, I) + cp.quad_form(u, L)), constraints)
	prob.solve()
	if prob.status == 'optimal': ds.append([(r - r_min)/r_var, (x_0-x_0_min)/x_0_var, (x_ref[0]-x_ref_min)/x_ref_var, u.value])
	

In [64]:
from pickle import dump
dump(ds, open("mpc.pkl", "wb"))

In [65]:
h = 0.01*np.random.rand() + 0.9

In [66]:
(h - 0.9)/0.01

0.6513486653710765

In [67]:
ds[0]

[0.7712550212968466,
 0.9093277461111476,
 0.2634844733706021,
 array([0.81814251, 0.66898224, 0.52365651, 0.37143429, 0.20107529])]

In [68]:
u.value

array([1.        , 1.        , 0.9314627 , 0.64182315, 0.34294526])

In [69]:
prob.status

'optimal'

In [70]:
lambda_ = torch.tensor(constraints[1].dual_value)

In [71]:
nu = torch.tensor(constraints[0].dual_value)

In [72]:
rho = torch.tensor(constraints[2].dual_value)

In [73]:
mu = torch.tensor(constraints[3].dual_value)

In [74]:
def grad_cost(U, x_t, x_ref, a, b, q, r):
	grad = torch.zeros_like(U)
	x = x_t
	for k in range(n):
		U_t = U[k].squeeze()
		x_next = a * x + b * U_t
		grad[k] = 2 * b * q * (x_next - x_ref[k].squeeze()) + 2 * r * U_t
		x = x_next
	return grad

In [75]:
def kkt_loss(U, lambda_, mu, nu, rho, x_t, x_ref, a, b, q, r):
		U_min = 0.0
		U_max = 1.0
		T_min = 0.0
		T_max = 1.0

		# Violazione della stazionarietà
		grad_J = grad_cost(U, x_t, x_ref, a, b, q, r)
		grad_ineq_control = lambda_ - mu
		grad_ineq_state = -b * nu + b * rho
		stationarity_violation = (grad_J + grad_ineq_control + grad_ineq_state).pow(2).sum()

		# Violazione della primal feasibility (controlli e stati)
		control_feasibility = (torch.max(U_min - U, torch.zeros_like(U)) + torch.max(U - U_max, torch.zeros_like(U))).pow(2).sum()

		x = copy(x_t)
		state_feasibility = 0
		for k in range(n):
			U_t = U[..., k].squeeze()
			x = copy(a * x + b * U_t)
			state_feasibility += (torch.max(T_min - x, torch.zeros_like(x)) + torch.max(x - T_max, torch.zeros_like(x))).pow(2).sum()

		# Violazione della complementarità
		comp_control = lambda_ * (U - U_min) + mu * (U_max - U)
		comp_state = 0
		x = copy(x_t)
		for k in range(n):
			U_t = U[k].squeeze()
			x_next = a * x + b * U_t
			nu_t = nu[k].squeeze()
			rho_t = rho[k].squeeze()
			comp_state += (nu_t * (T_min - x_next) + rho_t * (x_next - T_max)).pow(2).sum()
			x = copy(x_next)
		comp_loss = comp_control.pow(2).sum() + comp_state

		# Violazione della dual feasibility

		# Somma delle violazioni
		return grad_J, stationarity_violation, control_feasibility,state_feasibility,  comp_loss

In [76]:
kkt_loss(sol, lambda_, mu, nu, rho, t, t_ref, a, b, torch.tensor(1), pen)

NameError: name 'sol' is not defined

In [ ]:
grad = torch.zeros_like(sol)
x = copy(t)
for k in range(n):
	U_t = copy(sol[k].squeeze())
	x_next = a * x + b * U_t
	grad[k] = 2 * b  * (x_next - t_ref[k].squeeze()) + 2 * pen * U_t- b*nu[k]
	x = copy(x_next)

In [ ]:
lambda_

tensor([0., 0., 0., 0., 0.], dtype=torch.float64)

In [ ]:
sol

tensor([0.3827, 0.3616, 0.3281, 0.2713, 0.1726], dtype=torch.float64)

In [ ]:
grad

tensor([1.0003e-01, 9.0769e-02, 7.5044e-02, 4.7745e-02, 4.8572e-17],
       dtype=torch.float64)

In [ ]:
pen

0.17584272646532612

In [ ]:
sol[0]

tensor(0.3827, dtype=torch.float64)

In [ ]:
a * t + b * sol[0]

tensor(0.3484, dtype=torch.float64)

In [ ]:
t_ref[0].squeeze()

0.4382834245890117